### Implementation of Kalman Filter Approximation Techniques
As presented in the 1999 paper _"Super-resolution reconstruction of image sequences"_ by Michael Elad and Arie Feuer ([IEEE](https://ieeexplore.ieee.org/document/790425))

In [ ]:
%matplotlib inline

import numpy as np
from numpy.random import default_rng

from skimage import data, color
from skimage.transform import rescale
from skimage.util import random_noise
from skimage.filters import laplace

from scipy.signal import convolve2d

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [16, 9]

In [ ]:
def show_image(image):
    plt.imshow(image, cmap=plt.get_cmap("gray"))
    plt.colorbar()
    plt.show()

In [ ]:
# Algorithm params
k = 1 # number of image samples for video sequence
x_start_coord = 100 # start x position for frame k=0 
y_start_coord = 100 # start y position for frame k=0
x_shift_coord = 5 # x axis pixel shift distance between frames
y_shift_coord = 0 # y axis pixel shift distance between frames
hr_image_dim = 250 # assumed square
lr_image_dim = hr_image_dim // 2
img_dim_ratio = lr_image_dim / hr_image_dim

In [ ]:
# Original source image - scene
coffee = color.rgb2gray(data.coffee())
show_image(coffee)

# Synthetic video of 10 frames, based on cropped global shift of coffee image 
x_true = np.ndarray((k, hr_image_dim, hr_image_dim))

# generate frames of video stream
for frame in np.arange(k):
    x_clip_pos = x_start_coord + frame * x_shift_coord
    y_clip_pos = y_start_coord + frame * y_shift_coord
    x_true[frame] = coffee[y_clip_pos : y_clip_pos+hr_image_dim, x_clip_pos : x_clip_pos+hr_image_dim]
    show_image(x_true[frame])

In [ ]:
# filled to simulate a calculated 3x3 box blur
box_blur = np.full((3, 3), 1/9)

y_true = np.ndarray((k, lr_image_dim, lr_image_dim))

# Normal distribution noise parameters
mu = 0.0
sigma = 0.05
rng = default_rng() # needed for normal sampling

# generate low-resolution frames based on x_true frames
for frame in np.arange(k):
    intermediate = convolve2d(x_true[frame], box_blur, mode="same") # apply box blur
    intermediate = rescale(intermediate, img_dim_ratio, anti_aliasing=True) # decimate
    noise = mu + sigma * rng.normal(size=(lr_image_dim, lr_image_dim))
    y_true[frame] = intermediate + noise # apply additive noise
    show_image(y_true[frame])